In [ ]:

%pip install ipywidgets

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from ipywidgets import interact, Dropdown, Button, Output
from IPython.display import display, clear_output

# Init output area
out = Output()

# Create /charts folder
os.makedirs("charts", exist_ok=True)


In [ ]:

csv_files = [f for f in os.listdir() if f.endswith('.csv')]
if not csv_files:
    raise FileNotFoundError("No CSV files found in the directory.")

file_dropdown = Dropdown(
    options=csv_files,
    description='CSV File:',
    value=csv_files[0]
)

display(file_dropdown)


In [ ]:
# Define empty dropdowns (to be filled later)
x_dropdown = Dropdown(description='X-axis:')
y_dropdown = Dropdown(description='Y-axis:')
chart_dropdown = Dropdown(description='Chart Type:')
region_dropdown = Dropdown(description='Region:')
month_dropdown = Dropdown(description='Month:')
save_button = Button(description="💾 Save Chart")

display(x_dropdown, y_dropdown, chart_dropdown, region_dropdown, month_dropdown, save_button)


In [ ]:
def load_data(file_name):
    df = pd.read_csv(file_name)
    df.columns = df.columns.str.strip().str.title()
    return df

def update_widgets(change):
    global df
    df = load_data(change['new'])

    numeric_cols = df.select_dtypes(include='number').columns.tolist()
    categorical_cols = df.select_dtypes(exclude='number').columns.tolist()

    x_dropdown.options = df.columns.tolist()
    y_dropdown.options = df.columns.tolist()
    chart_dropdown.options = ['line', 'bar', 'scatter', 'box', 'hist', 'pie']

    if 'Region' in df.columns:
        region_dropdown.options = ['All'] + sorted(df['Region'].dropna().unique().tolist())
    else:
        region_dropdown.options = ['All']

    if 'Month' in df.columns:
        month_dropdown.options = ['All'] + sorted(df['Month'].dropna().unique().tolist())
    else:
        month_dropdown.options = ['All']

file_dropdown.observe(update_widgets, names='value')
update_widgets({'new': file_dropdown.value})  # Trigger once on load


In [ ]:
#Chart Type & Axis Selectors

x_dropdown = Dropdown(description='X-axis:')
y_dropdown = Dropdown(description='Y-axis:')
chart_dropdown = Dropdown(
    options=['line', 'bar', 'scatter', 'box', 'hist', 'pie'],
    value='bar',
    description='Chart Type:'
)

region_dropdown = Dropdown(description='Region:')
month_dropdown = Dropdown(description='Month:')

save_button = Button(description="💾 Save Chart")
display(x_dropdown, y_dropdown, chart_dropdown, region_dropdown, month_dropdown, save_button)


In [ ]:
# Plotting Logic

def filter_df():
    filtered = df.copy()
    if region_dropdown.value != 'All':
        filtered = filtered[filtered['Region'] == region_dropdown.value]
    if month_dropdown.value != 'All':
        filtered = filtered[filtered['Month'] == month_dropdown.value]
    return filtered

def plot_chart(x, y, chart_type):
    filtered = filter_df()
    plt.figure(figsize=(10, 6))
    if chart_type == 'bar':
        sns.barplot(data=filtered, x=x, y=y)
    elif chart_type == 'line':
        sns.lineplot(data=filtered, x=x, y=y)
    elif chart_type == 'scatter':
        sns.scatterplot(data=filtered, x=x, y=y)
    elif chart_type == 'box':
        sns.boxplot(data=filtered, x=x, y=y)
    elif chart_type == 'hist':
        sns.histplot(data=filtered, x=y, bins=20, kde=True)
    elif chart_type == 'pie':
        pie_data = filtered.groupby(x)[y].sum()
        pie_data.plot(kind='pie', autopct='%1.1f%%', startangle=90)
        plt.ylabel('')
    else:
        print("Unknown chart type")
    plt.title(f"{chart_type.title()} - {y} vs {x}")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def on_change(change):
    if x_dropdown.value and y_dropdown.value and chart_dropdown.value:
        with out:
            clear_output(wait=True)
            plot_chart(x_dropdown.value, y_dropdown.value, chart_dropdown.value)

x_dropdown.observe(on_change, names='value')
y_dropdown.observe(on_change, names='value')
chart_dropdown.observe(on_change, names='value')
region_dropdown.observe(on_change, names='value')
month_dropdown.observe(on_change, names='value')
display(out)


In [ ]:
#Save Chart Handler

def on_save_clicked(b):
    filtered = filter_df()
    fig, ax = plt.subplots(figsize=(10, 6))
    chart_type = chart_dropdown.value
    x, y = x_dropdown.value, y_dropdown.value

    if chart_type == 'bar':
        sns.barplot(data=filtered, x=x, y=y, ax=ax)
    elif chart_type == 'line':
        sns.lineplot(data=filtered, x=x, y=y, ax=ax)
    elif chart_type == 'scatter':
        sns.scatterplot(data=filtered, x=x, y=y, ax=ax)
    elif chart_type == 'box':
        sns.boxplot(data=filtered, x=x, y=y, ax=ax)
    elif chart_type == 'hist':
        sns.histplot(data=filtered, x=y, bins=20, kde=True, ax=ax)
    elif chart_type == 'pie':
        pie_data = filtered.groupby(x)[y].sum()
        pie_data.plot(kind='pie', autopct='%1.1f%%', startangle=90, ax=ax)
        ax.set_ylabel('')

    ax.set_title(f"{chart_type.title()} - {y} vs {x}")
    plt.xticks(rotation=45)
    plt.tight_layout()

    filename = f"charts/{x}_{y}_{chart_type}.png".replace(" ", "_")
    fig.savefig(filename)
    print(f"✅ Chart saved as {filename}")
    plt.close(fig)

save_button.on_click(on_save_clicked)
